In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import os

In [ ]:
# Send a GET request to the URL
url_topic = "https://medium.com/tag/llm/archive"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
    
response = requests.get(url_topic, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all 'a' tags and extract the 'href' attribute
    hrefs = [a.get('href') for a in soup.find_all('a') if a.get('href')]
    hrefs = list(set([h for h in hrefs if "@" in h]))

else:
    print(f"Failed to retrieve the web page. Status code: {response.status_code}")
    


for link in hrefs:
    print(link)

In [ ]:
posts

In [ ]:
article

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import re

from langdetect import detect


def extract_post_timing(article):
    divs = article.find_all("div")
    div_texts = [x.get_text(strip=True) for x in divs]
    when = [x for x in div_texts if isinstance(x, str)]
    when = [replace_just_now(x) for x in when]

    when_ = []
    for x in when:
        m = extract_time_ago(x)
        if len(m) > 0:
            when_ += m

    when = list(set(when_))
    if len(when) > 0:
        when = when[0]
    else:
        print(f"Failed when ....{div_texts}\n")
        when = None
        
    return when

def extract_time_ago(input_string):
    # Define the regex pattern to match the time format
    pattern = r'\b\d{1,2}[hd] ago'
    # Find all matches in the input string
    matches = re.findall(pattern, input_string)
    return matches


def replace_just_now(input_str):
    if input_str.lower() == "just now":
        return "0h ago"
    return input_str


def extract_links(this_article):
    # extract link
    links = [a.get('data-href') for a in this_article.find_all('div') if a.get('data-href')]
    if len(links) > 0:
        return links[0]
    else:
        print(f"Failed links ....{links}\n")
        return None


def extract_snippet(this_article):
    # extract short text
    h3s = this_article.find_all('h3')
    snippet = [h3.get_text(strip=True) for h3 in h3s]
    if len(snippet) > 0:
        return snippet[0]
    else:
        print(f"Failed snippet ....{links}\n")
        None


def extract_title(this_article):
        # extract title
    h2s = this_article.find_all('h2')
    titles = [h2.get_text(strip=True) for h2 in h2s]
    
    titles_ = [x.get("aria-label") for x in this_article.find_all('div')]
    titles_ = [x for x in titles_ if isinstance(x, str)]
    if len(titles_) > 0:
        title = titles_[0]
    elif len(titles) > 0:
        title = titles[0]
    else:
        print(f"Failed links ....{titles_}\n")
        title = None

    return title

def extract_image(this_article):
    srcs = list(set([x.get("src") for x in this_article.find_all('img')]))
    imgs = [img for img in srcs if ".jpg" in img.lower() or ".png" in img.lower() or ".jpeg" in img.lower()]
    if len(imgs) > 0:
        return imgs[0]
    else:
        print(f"Failed links ....{srcs}\n")
        return None
    return 

def parsing_article(this_article):

    link = extract_links(this_article)
    title = extract_title(this_article)
    when = extract_post_timing(this_article)
    img = extract_image(this_article)
    snippet = extract_snippet(this_article)
    post = {"title": title, "link": link, "img": img, "snippet": snippet, "when": when}

    return post


# Helper function to check if the timestamp is longer than 1 day ago
def is_longer_than_1_day(timestamp):
    if 'd ago' in timestamp:
        return True
    return False

# Initialize the WebDriver with headless mode
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Initialize the WebDriver
driver = webdriver.Chrome(options=options)

# URL of the Medium tag page
url = "https://medium.com/tag/llm/archive"
driver.get(url)

# Wait until the page is fully loaded
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

# # Define a function to extract articles
def extract_articles(driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.find_all('article')
    article_data = []
    for this_article in articles:
        post = parsing_article(this_article)
        if detect(post["title"]) != "en":
            continue
        article_data.append(post)
    return article_data






# Scroll the page and collect articles
collected_articles = []
run = True
cnt = 1
while run:  # Condition to stop scrolling (e.g., 50 articles)
    articles = extract_articles(driver)
    for article in articles:
        ts = article["when"]
        print(article['title'], ts)
        if is_longer_than_1_day(ts):
           run = False
        else:
            collected_articles.append(article)

    # Scroll to the bottom of the page
    if len(collected_articles) == 0 and cnt == 1:
        run = False
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    cnt += 1
    time.sleep(2)  # Wait for new content to load
    run = False
driver.quit()


Enhancing Mathematical Reasoning in AI: Integrating LLMs with Monte Carlo Tree Search 11h ago
Refining the Role of GPT-4 LLM in Virtual Internships 11h ago
Literature Review Generation using Llama and Arxiv 12h ago
Simplify LLM Quantization Process for Success 12h ago
RAG : Q/A from video content using Lang-chain and Mistral-7B 13h ago
【学习LangChain】02. 记忆（Memory） 13h ago
Master LLM Sentiment Analysis: A Simple Guide 13h ago


In [6]:
collected_articles

[{'title': 'Enhancing Mathematical Reasoning in AI: Integrating LLMs with Monte Carlo Tree Search',
  'link': 'https://medium.com/@chaudharysahil379/enhancing-mathematical-reasoning-in-ai-integrating-llms-with-monte-carlo-tree-search-b3ef188cba9a',
  'img': 'https://miro.medium.com/v2/resize:fill:160:107/1*rXEX6IifVU0bhhKBQPBWpQ.png',
  'snippet': 'Introduction',
  'when': '11h ago'},
 {'title': 'Refining the Role of GPT-4 LLM in Virtual Internships',
  'link': 'https://medium.com/@rishitmayank/refining-the-role-of-gpt-4-llm-in-virtual-internships-f4bdb5798ea3',
  'img': 'https://miro.medium.com/v2/resize:fill:80:53/1*7xaWTdSG9NEJisj2TVdNzg.jpeg',
  'snippet': 'The deployment of GPT-4 LLM as a virtual intern not only streamlines mundane tasks but also enriches the learning experience for human…',
  'when': '11h ago'},
 {'title': 'Literature Review Generation using Llama and Arxiv',
  'link': 'https://medium.com/@khaoujai/literature-review-generation-using-llama-and-arxiv-8a25aecb3f04',

In [11]:
def valide_post(article, subtopics=[]):
    link = article["link"]
    if instance(link, type(None)):
        return False
    
    options = webdriver.ChromeOptions()
    #options.add_argument('--headless=new')
    options.add_argument('--disable-gpu')
    #options.add_argument('--no-sandbox')
    #options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    driver.get(link)

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tags = soup.find_all('href')
    tags = [tag.get('href') for tag in tags]
    tags = soup.find_all('a', href=True)
    tags = [tag.get('href') for tag in tags]
    tags = [tag for tag in tags if "/tag/" in tag]
    tags = [tag.split("?source=post_page")[0].replace("/tag/", "") for tag in tags]
    match_tags = [tag for tag in tags if tag in subtopics]

    return len(match_tags) > 0

[]


In [21]:
tags = soup.find_all('a', href=True)
tags = [tag.get('href') for tag in tags]
tags = [tag for tag in tags if "/tag/" in tag]
tags = [tag.split("?source=post_page")[0].replace("/tag/", "") for tag in tags]
tags

['llm', 'mathematical-reasoning']

In [ ]:
article